In [ ]:
import os
import random
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
import shutil
import kagglehub

In [ ]:
# Step 1: Download Dataset
path = kagglehub.dataset_download("shubhamgoel27/dermnet")
print("Dataset downloaded to:", path)

train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")
train_target_dir = "/distributed_train"
test_target_dir = "/distributed_test"

100%|██████████| 1.72G/1.72G [00:21<00:00, 84.2MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/shubhamgoel27/dermnet/versions/1


In [ ]:
# Copy dataset to a simpler location in the working directory
new_path = "/content/dermnet_dataset"
!cp -r {path} {new_path}

# Update train and test paths
train_dir = os.path.join(new_path, "train")
test_dir = os.path.join(new_path, "test")

print("New Train Directory:", train_dir)
print("New Test Directory:", test_dir)


New Train Directory: /content/dermnet_dataset/train
New Test Directory: /content/dermnet_dataset/test


In [ ]:
import os
import pandas as pd

# Paths for train and test directories
train_dir = "/content/dermnet_dataset/train"
test_dir = "/content/dermnet_dataset/test"

# Selected classes
selected_classes = [
    "Psoriasis pictures Lichen Planus and related diseases",
    "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Melanoma Skin Cancer Nevi and Moles",
    "Nail Fungus and other Nail Disease",
    "Acne and Rosacea Photos",
    "Warts Molluscum and other Viral Infections",
    "Seborrheic Keratoses and other Benign Tumors"
]

# Function to simplify class names
def simplify_class_name(class_name):
    # Define a mapping for simplification
    simplification_mapping = {
        "Psoriasis pictures Lichen Planus and related diseases": "Psoriasis",
        "Tinea Ringworm Candidiasis and other Fungal Infections": "Fungal Infections",
        "Melanoma Skin Cancer Nevi and Moles": "Melanoma",
        "Nail Fungus and other Nail Disease": "Nail Fungus",
        "Acne and Rosacea Photos": "Acne",
        "Warts Molluscum and other Viral Infections": "Warts",
        "Seborrheic Keratoses and other Benign Tumors": "Benign Tumors"
    }
    return simplification_mapping.get(class_name, class_name)  # Default to original if not mapped

# Function to create a DataFrame for a given directory
def create_dataframe(base_dir, selected_classes):
    data = []
    for class_name in selected_classes:
        class_path = os.path.join(base_dir, class_name)
        if os.path.exists(class_path):
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                simplified_class_name = simplify_class_name(class_name)
                data.append({"image_path": image_path, "label": class_name, "class_name": simplified_class_name})
    return pd.DataFrame(data)

# Create DataFrames for train and test folders
train_df = create_dataframe(train_dir, selected_classes)
test_df = create_dataframe(test_dir, selected_classes)

# Print the head of both DataFrames
print("Train DataFrame Head:")
print(train_df.head())

print("\nTest DataFrame Head:")
print(test_df.head())


Train DataFrame Head:
                                          image_path  \
0  /content/dermnet_dataset/train/Psoriasis pictu...   
1  /content/dermnet_dataset/train/Psoriasis pictu...   
2  /content/dermnet_dataset/train/Psoriasis pictu...   
3  /content/dermnet_dataset/train/Psoriasis pictu...   
4  /content/dermnet_dataset/train/Psoriasis pictu...   

                                               label class_name  
0  Psoriasis pictures Lichen Planus and related d...  Psoriasis  
1  Psoriasis pictures Lichen Planus and related d...  Psoriasis  
2  Psoriasis pictures Lichen Planus and related d...  Psoriasis  
3  Psoriasis pictures Lichen Planus and related d...  Psoriasis  
4  Psoriasis pictures Lichen Planus and related d...  Psoriasis  

Test DataFrame Head:
                                          image_path  \
0  /content/dermnet_dataset/test/Psoriasis pictur...   
1  /content/dermnet_dataset/test/Psoriasis pictur...   
2  /content/dermnet_dataset/test/Psoriasis pictur...   

In [ ]:
train_df["class_name"].value_counts()

,count
class_name,
Psoriasis,1405
Benign Tumors,1371
Fungal Infections,1300
Warts,1086
Nail Fungus,1040
Acne,840
Melanoma,463


In [ ]:
test_df["class_name"].value_counts()

,count
class_name,
Psoriasis,352
Benign Tumors,343
Fungal Infections,325
Acne,312
Warts,272
Nail Fungus,261
Melanoma,116


In [ ]:
#updated code to efficientnetb1

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import time
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.efficientnet_b1(pretrained=False)

for param in model.features.parameters():
    param.requires_grad = False

num_classes = len(selected_classes)  # 7 classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

#training and validation loops
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    #progress bar
    with tqdm(train_loader, unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description(f"Epoch {epoch + 1}")
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total

            tepoch.set_postfix(loss=running_loss / (total / train_loader.batch_size), accuracy=accuracy)

    return running_loss / len(train_loader), accuracy


def validate(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            #forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(test_loader), accuracy


#training with checkpoints
num_epochs = 30
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
best_val_loss = float("inf")
checkpoint_path = "best_model_checkpoint.pth"

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    #training
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, epoch)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    #validation
    val_loss, val_accuracy = validate(model, test_loader, criterion)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Save model checkpoint if loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), checkpoint_path)
        print(f"New best model saved as '{checkpoint_path}'")

print("\nTraining complete. Best model saved as 'best_model_checkpoint.pth'")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/30


Epoch 1: 100%|██████████| 308/308 [02:36<00:00,  1.97batch/s, accuracy=68.9, loss=0.78]


Validation Loss: 0.7588, Validation Accuracy: 70.65%
New best model saved as 'best_model_checkpoint.pth'

Epoch 2/30


...

Validation Loss: 0.6008, Validation Accuracy: 80.02%

Epoch 8/30


Epoch 8: 100%|██████████| 308/308 [02:39<00:00,  1.94batch/s, accuracy=89.7, loss=0.243]


Validation Loss: 0.7174, Validation Accuracy: 77.70%

Epoch 9/30


Epoch 9:  44%|████▍     | 137/308 [01:11<01:29,  1.91batch/s, accuracy=90.8, loss=0.226]


KeyboardInterrupt: 